# Biking Prediction with Neural Networks

### Overview - In previous work a neural network was written from scratch to translate biking times into distance. It is the goal of this interactive notebook to supercharge my previous project by providing direct access to Strava data through V3 of their API and a TensorFlow Implementation of a feed-forward neural network.

In [1]:
#Import Dependicies and setup local config
import stravalib
import BaseHTTPServer
import webbrowser
import tensorflow as tf

with open('client.secret') as f:
    client_id, secret = f.read().strip().split(',')
port = 8080
url = 'http://localhost:%d/authorized' % port


In [ ]:
#Establish a webservice to access data through
client = stravalib.client.Client()
authorize_url = client.authorization_url(client_id=client_id, redirect_uri = url)
webbrowser.open(authorize_url)


In [ ]:
#Establish Neural Network Model Architecture

#Pulled directly from Tensor-Ops lib
def affine(x, output_dim, scope=None, stddev=1.0, const=0.1):
    """
    Fully connected affine layer with weight and bias initialized using inputs or defaults
    :param x: Input data to layer
    :param output_dim: Desired output layer nodes
    :param scope: (Optional) Name to scope the variable with
    :param stddev: (Optional) Standard Deviation to initialize weights with
    :param const: (Optional) Constant value to initialize biases with
    :return: Output fully-connected layer
    """
    with tf.variable_scope(scope or 'linear'):
        w = tf.get_variable('W', [x.get_shape()[1], output_dim], tf.float32,
                            initializer=tf.truncated_normal_initializer(stddev=stddev))
        b = tf.get_variable('b', [output_dim], tf.float32, initializer=tf.constant_initializer(const))
        return tf.matmul(x, w) + b

#Hyperparameters
feature_size = 5
h_dim = 100
dropout_percent = 0.5

#Training Config
config = {}
config['epoch'] = 5
config['bs'] = 15
config['lr'] = 5e-4
config['lam'] = 0.5
config['b1'] = 0.5

#Directories
checkpoint_dir = './checkpoints/'
log_dir = './logs/'

#Input and Optimization Function
x = tf.placeholder(tf.float32, shape=[None, feature_size], name='input')
    
with tf.variable_scope('model'):
    #Pass through a 5 layer NN
    h1 = affine(x, h_dim, 'h1')
    a1 = tf.nn.relu(h1)
    h2 = affine(a1, h_dim, 'h2')
    a2 = tf.nn.relu(h2)
    h3 = affine(a2, h_dim, 'h3')
    a3 = tf.nn.relu(h3)
    #Dropout
